# Modeling

In [ ]:
import fasttext
import fasttext.util
import numpy as np
import pandas as pd
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report,
    confusion_matrix,
)
from sklearn.model_selection import train_test_split

# Either load the bin file using the command, or the text (vector) file from https://fasttext.cc/docs/en/crawl-vectors.html
# fasttext.util.download_model("de", if_exists="ignore")

## FastText

---

In [ ]:
parameter = {"input": "train.txt", "pretrainedVectors": "cc.de.300.vec"} # "epoch": 50, "lr": 0.05, "wordNgrams": 2, "verbose": 2, "minCount":1, "loss": "ns", "lrUpdateRate": 100, "thread": 4, "ws": 5, "dim": 300,


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    df_modeling["clean_text"], df_modeling["party"], test_size=0.2, random_state=42
)


In [ ]:
df_test = pd.DataFrame({"text": X_test, "party": y_test})

In [ ]:
test = df_modeling.copy()

with open(Path("train.txt"), "w") as f:
    for index, row in test[:int(test.shape[0] * 0.8)].iterrows():
        f.write(f"__label__{row['party']} {row['clean_text']}\n")

with open(Path("test.txt"), "w") as f:
    for index, row in test[int(test.shape[0] * 0.8):].iterrows():
        f.write(f"__label__{row['party']} {row['clean_text']}\n")


In [ ]:
model = fasttext.train_supervised(input="train.txt", epoch=5, lr=0.1, wordNgrams=2, loss="softmax", dim=300, pretrainedVectors="cc.de.300.vec")

In [ ]:
test_score = model.test("test.txt")

print(f"Count of test data (N): {test_score[0]}")
print(f"F1 Score: {2 * ((test_score[1] * test_score[2]) / (test_score[1] + test_score[2]))}")
print(f"Percision: {test_score[1]}")
print(f"Recall: {test_score[2]}")

In [ ]:
df_test["prediction"] = df_test["text"].apply(lambda x: int(model.predict(x)[0][0].replace("__label__", "")))

In [ ]:
cm = confusion_matrix(df_test["party"], df_test["prediction"], normalize="true")
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=party_encoding.keys())
disp.plot()